<a href="https://colab.research.google.com/github/kungfumas/stbi/blob/master/search_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup# Make a request to the website
r = requests.get('https://bola.kompas.com/')# Create an object to parse the HTML format
soup = BeautifulSoup(r.content, 'html.parser')# Retrieve all popular news links (Fig. 1)
link = []
for i in soup.find('div', {'class':'most__wrap'}).find_all('a'):
    i['href'] = i['href'] + '?page=all'
    link.append(i['href'])# For each link, we retrieve paragraphs from it, combine each paragraph as one string, and save it to documents (Fig. 2)
documents = []
for i in link:
    # Make a request to the link
    r = requests.get(i)
  
    # Initialize BeautifulSoup object to parse the content 
    soup = BeautifulSoup(r.content, 'html.parser')
  
    # Retrieve all paragraphs and combine it as one
    sen = []
    for i in soup.find('div', {'class':'read__content'}).find_all('p'):
        sen.append(i.text)
  
    # Add the combined paragraphs to documents
    documents.append(' '.join(sen))

In [25]:
import re
documents_clean = []
for d in documents:
    # Remove Unicode
    document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    # Lowercase the document
    document_test = document_test.lower()
    # Remove punctuations
    #document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
    # Lowercase the numbers
    document_test = re.sub(r'[0-9]', '', document_test)
    # Remove the doubled space
    document_test = re.sub(r'\s{2,}', ' ', document_test)
    documents_clean.append(document_test)

In [28]:
documents_clean[0:5]

['kompas.com - menpora zainudin amali menilai pesepak bola yang tidak bisa bersikap profesional dan menjunjung tinggi sportivitas tidak layak memperkuat timnas indonesia. pendapat itu diungkapkan zainudin amali setelah mengetahui aksi brutal dua pemain ahha ps pati fc pada laga uji coba melawan persiraja banda aceh. duel ahha ps pati fc vs persiraja berlangsung di pancoran soccer field pada senin (//). meski berstatus uji coba, laga ahha ps pati vs persiraja berlangsung sangat keras sampai terjadi keributan antar pemain. dua momen yang paling disorot dari laga tersebut adalah pelanggaran keras yang dilakukan dua pemain ahh ps pati, yakni syaiful indra cahya dan zulham zamrun. baca juga: soal aksi brutal di laga ahha ps pati vs persiraja, menpora minta pssi turun tangan syaiful indra cahya mendapatkan kartu merah setelah melepaskan tendangan kungfu yang mengenai wajah pemain persiraja, muhammad nadhiif. dapatkan informasi, inspirasi dan insight di email kamu.daftarkan email di sisi lain

In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
# Instantiate a TfidfVectorizer object
vectorizer = TfidfVectorizer()# It fits the data and transform it as a vector
X = vectorizer.fit_transform(documents_clean)# Convert the X as transposed matrix
X = X.T.toarray()# Create a DataFrame and set the vocabulary as the index
df = pd.DataFrame(X, index=vectorizer.get_feature_names())

In [19]:
df.head()

,0,1,2,3,4,5,6,7,8,9
abdul,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.046924,0.0,0.0,0.000000
abe,0.000000,0.000000,0.000000,0.031505,0.000000,0.0000,0.000000,0.0,0.0,0.000000
absen,0.000000,0.000000,0.038069,0.000000,0.000000,0.0000,0.000000,0.0,0.0,0.059627
ac,0.000000,0.000000,0.000000,0.000000,0.046493,0.0000,0.000000,0.0,0.0,0.000000
aceh,0.065409,0.051883,0.000000,0.000000,0.000000,0.0279,0.124111,0.0,0.0,0.000000


In [20]:
df.tail()

,0,1,2,3,4,5,6,7,8,9
zidane,0.000000,0.000000,0.0,0.00000,0.023247,0.000000,0.000000,0.0,0.0,0.0
zinedine,0.000000,0.000000,0.0,0.00000,0.023247,0.000000,0.000000,0.0,0.0,0.0
zlatan,0.000000,0.000000,0.0,0.00000,0.023247,0.000000,0.000000,0.0,0.0,0.0
zona,0.000000,0.000000,0.0,0.06301,0.000000,0.000000,0.000000,0.0,0.0,0.0
zulham,0.109015,0.103766,0.0,0.00000,0.000000,0.167397,0.031028,0.0,0.0,0.0


In [24]:
def get_similar_articles(q, df):
  print("query:", q)
  print("Berikut artikel dengan nilai cosine similarity tertinggi: ")  # Convert the query become a vector
  q = [q]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  sim = {}  # Calculate the similarity
  for i in range(10):
    sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  
  # Sort the values 
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)  # Print the articles and their similarity values
  for k, v in sim_sorted:
    if v != 0.0:
      print("Nilai Similaritas:", v)
      print(documents_clean[k])
      print()# Add The Query
q1 = 'barcelona'# Call the function
get_similar_articles(q1, df)

query: barcelona
Berikut artikel dengan nilai cosine similarity tertinggi: 
Nilai Similaritas: 0.08279099439190626
kompas.com - comeback louis van gaal bersama timnas belanda terus menuai kesuksesan. terkini, virgil van dijk cs menang besar - atas turki pada rabu (//). media asal belanda, algemein dagblaad (ad), mengatakan "kehadiran louis van gaal mendatangkan atmosfer yang benar-benar berbeda dalam waktu satu minggu". bagaimana tidak, kemenangan besar atas turki menutup jendela internasional produktif bagi timnas belanda dan van gaal. kendati memulai pekan dengan hasil seri - lawan norwegia, belanda lalu memukul turki - sebelum mencatatkan kemenangan - lawan turki ini. penyerang barcelona memphis depay jadi motor kemenangan barcelona di stadion johan cruyff tersebut. baca juga: hasil perancis vs finlandia - griezmann gemilang, les bleus putus tren negatif sang bomber mencetak hat-trick (', 'pen', ') dengan gol-gol dari davy klaasen ('), guus til ('), dan donyell malen (') melengkapi 